In [1]:
import os
import zipfile
import time
import requests

import pandas as pd
import numpy as np
import fiona
import geopandas as gpd

from sqlalchemy import create_engine

# set up database
user = os.environ.get('DB_USERNAME')
pwd = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOSTNAME')
port = os.environ.get('DB_PORT')

db_connection_url = f"postgres+psycopg2://{user}:{pwd}@{host}:{port}/geodemo"
engine = create_engine(db_connection_url)

In [3]:
zfs = [f for f in os.listdir('../data') if f.split('.')[-1] == 'zip']

In [4]:
zfs

['OpenNames_gpkg.zip',
 'BoundaryLine_gpkg.zip',
 'OpenMapLocal_gpkg.zip',
 'OpenGreenspace_gpkg.zip',
 'OpenRoads_gpkg.zip',
 'OpenUSRN_gpkg.zip',
 'CodePointOpen_gpkg.zip',
 'OpenZoomstack_gpkg.zip',
 'OpenTOID_gpkg.zip',
 'Strategi.zip',
 'OpenRivers_gpkg.zip',
 'OpenUPRN_gpkg.zip']

In [5]:
layer_lists = dict()

In [6]:
# list layers
for f in zfs:
    
    try:
        zf = zipfile.ZipFile(os.path.join('..','data',f))
        [gpkg] =  [f for f in zf.namelist() if f.split('.')[-1] == 'gpkg']
        print(gpkg)
        layers = fiona.listlayers(zf.open(gpkg))
        print(len(layers))
        print(layers)
        layer_lists[zf] = layers
    except Exception as e:
        print(e)

data/opname_gb.gpkg
1
['NamedPlace']
data/bdline_gb.gpkg
18
['boundary_line_ceremonial_counties', 'boundary_line_historic_counties', 'community_ward', 'country_region', 'county', 'county_electoral_division', 'district_borough_unitary', 'district_borough_unitary_ward', 'english_region', 'greater_london_const', 'high_water', 'historic_european_region', 'parish', 'polling_districts_england', 'scotland_and_wales_const', 'scotland_and_wales_region', 'unitary_electoral_division', 'westminster_const']
data/opmplc_gb.gpkg
20
['Building', 'CarChargingPoint', 'ImportantBuilding', 'NamedPlace', 'Road', 'Foreshore', 'FunctionalSite', 'SurfaceWater_Line', 'SurfaceWater_Area', 'TidalWater', 'TidalBoundary', 'Woodland', 'Glasshouse', 'ElectricityTransmissionLine', 'RailwayStation', 'RailwayTrack', 'RailwayTunnel', 'RoadTunnel', 'Roundabout', 'MotorwayJunction']
OS Open Greenspace (GPKG) GB/data/opgrsp_gb.gpkg
2
['AccessPoint', 'GreenspaceSite']
data/oproad_gb.gpkg
3
['RoadLink', 'RoadNode', 'Motorway

In [13]:
sql = '''SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';'''

tables1 = pd.read_sql(sql, engine).table_name.values

In [14]:
tables1

array(['spatial_ref_sys', 'opennames', 'openrivers', 'openroads',
       'boundaryline', 'openmaplocal', 'opengreenspace', 'openuprn',
       'opentoid', 'boundary_line_ceremonial_counties',
       'boundary_line_historic_counties', 'community_ward',
       'country_region', 'county', 'county_electoral_division',
       'district_borough_unitary', 'district_borough_unitary_ward',
       'english_region', 'greater_london_const', 'high_water',
       'historic_european_region', 'parish', 'polling_districts_england',
       'scotland_and_wales_const', 'scotland_and_wales_region',
       'unitary_electoral_division', 'westminster_const',
       'code_point_open'], dtype=object)

In [10]:
tables

array(['spatial_ref_sys', 'opennames', 'openrivers', 'openroads',
       'boundaryline', 'openmaplocal', 'opengreenspace', 'openuprn',
       'opentoid', 'boundary_line_ceremonial_counties',
       'boundary_line_historic_counties', 'community_ward',
       'country_region', 'county', 'county_electoral_division',
       'district_borough_unitary', 'district_borough_unitary_ward',
       'english_region', 'greater_london_const', 'high_water',
       'historic_european_region', 'parish', 'polling_districts_england',
       'scotland_and_wales_const', 'scotland_and_wales_region',
       'unitary_electoral_division', 'westminster_const'], dtype=object)

In [15]:
# unpack all layers to postgis

for f in zfs:
#     print(f)
#     if f.split('_')[0].lower() in tables:
#         print(f'{f} in tables')
#     else:
        try:

            t0 = time.time()
            zf = zipfile.ZipFile(os.path.join('..','data',f))
            [gpkg] = [f for f in zf.namelist() if f.split('.')[-1] == 'gpkg']
            print(gpkg)
            layers = fiona.listlayers(zf.open(gpkg))
            print(len(layers))
            print(layers)

            t1 = time.time()
            print(t1-t0)

            if len(layers)>1:
                for l in layers[1:]:

                    if l in tables:
                        print(f'{l} in tables')
                    else:
                        print(l)
                        gdf = gpd.read_file(zf.open(gpkg), driver='GPKG', layer=layers[0])
                        t2 = time.time()
                        print(t2-t1)
                        gdf.to_postgis(l, engine)
                        t3 = time.time()
                        print(t3-t2)

        except Exception as e:
            print(e)

data/opname_gb.gpkg
1
['NamedPlace']
7.857110023498535
data/bdline_gb.gpkg


KeyboardInterrupt: 

In [34]:
%%time
print(1)
%%time
print(2)

1


UsageError: Line magic function `%%time` not found.


In [32]:
# unzip docs
for f in zfs:
    try:
        print(f)
        
        fname = f.split('_')[0].split('.')[0]
        fpath = os.path.join('..','data',fname)
        if not os.path.exists(fpath):
            os.mkdir(fpath)
        zf = zipfile.ZipFile(os.path.join('..','data',f))
        names = [f for f in zf.namelist() if f.split('.')[-1] != 'gpkg']
        for n in names:
            nname = n.split('/')[-1]
            print(f'"{nname}"')
            if nname:
                with open(os.path.join(fpath,nname),'wb') as unzip:
                    unzip.write(zf.open(n).read())
                    unzip.close()
    except Exception as e:
        print(e)

OpenNames_gpkg.zip
"readme.txt"
"licence.txt"
BoundaryLine_gpkg.zip
""
""
"B-L_COMMUNITY_WARD_README.txt"
"Ceremonial_Counties.docx"
"Counties - circa 1888-1889-Acts.docx"
"ukpga_20110001_en.pdf"
"readme.txt"
OpenMapLocal_gpkg.zip
"readme.txt"
"licence.txt"
OpenGreenspace_gpkg.zip
"readme.txt"
"licence.txt"
OpenRoads_gpkg.zip
"readme.txt"
"licence.txt"
OpenUSRN_gpkg.zip
"osopenusrn_202108_versions.json"
"licence.txt"
CodePointOpen_gpkg.zip
"readme.txt"
"Code-Point_Open_Column_Headers.csv"
"Codelist.xlsx"
"licence.txt"
"metadata.txt"
"NHS_Codelist.xls"
OpenZoomstack_gpkg.zip
File is not a zip file
OpenTOID_gpkg.zip
"osopentoid_202108_versions.json"
"licence.txt"
Strategi.zip
""
""
"admin_line.dbf"
"admin_line.prj"
"admin_line.shp"
"admin_line.shx"
"admin_seed.dbf"
"admin_seed.prj"
"admin_seed.shp"
"admin_seed.shx"
"antiquity_line.dbf"
"antiquity_line.prj"
"antiquity_line.shp"
"antiquity_line.shx"
"a_road.dbf"
"a_road.prj"
"a_road.shp"
"a_road.shx"
"b_road.dbf"
"b_road.prj"
"b_road.shp"


In [21]:
zf = zipfile.ZipFile('../data/BoundaryLine_gpkg.zip')

In [23]:
zf.namelist()

['data/',
 'data/bdline_gb.gpkg',
 'doc/',
 'doc/B-L_COMMUNITY_WARD_README.txt',
 'doc/Ceremonial_Counties.docx',
 'doc/Counties - circa 1888-1889-Acts.docx',
 'doc/ukpga_20110001_en.pdf',
 'readme.txt']

In [24]:
zf.open('readme.txt').

<zipfile.ZipExtFile name='readme.txt' mode='r' compress_type=deflate>

In [18]:
os.getcwd()

'/home/jovyan/code'